# 纠错 RAG (CRAG) 实现

在这个笔记本中，我实现了纠错 RAG - 一种先进的方法，它动态评估检索到的信息，并在必要时纠正检索过程，使用网络搜索作为后备方案。

CRAG 通过以下方式改进传统 RAG：

- 在使用检索内容之前对其进行评估
- 根据相关性动态切换知识源
- 当本地知识不足时，通过网络搜索纠正检索
- 在适当时结合来自多个源的信息

## 设置环境
我们首先导入必要的库。

In [ ]:
import os
import numpy as np
import json
import fitz  # PyMuPDF
from openai import OpenAI
import requests
from typing import List, Dict, Tuple, Any
import re
from urllib.parse import quote_plus
import time

## 设置 OpenAI API 客户端
我们初始化 OpenAI 客户端来生成嵌入向量和响应。

In [ ]:
# 使用基础 URL 和 API 密钥初始化 OpenAI 客户端
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 从环境变量中获取 API 密钥
)

## 文档处理函数

In [ ]:
def extract_text_from_pdf(pdf_path):
    """
    从 PDF 文件中提取文本内容。
    
    Args:
        pdf_path (str): PDF 文件路径
        
    Returns:
        str: 提取的文本内容
    """
    print(f"正在从 {pdf_path} 提取文本...")
    
    # 打开 PDF 文件
    pdf = fitz.open(pdf_path)
    text = ""
    
    # 遍历 PDF 中的每一页
    for page_num in range(len(pdf)):
        page = pdf[page_num]
        # 从当前页面提取文本并追加到 text 变量中
        text += page.get_text()
    
    return text

In [ ]:
def chunk_text(text, chunk_size=1000, overlap=200):
    """
    将文本分割为重叠的块，以便高效检索和处理。
    
    此函数将大文本分割为较小的、可管理的块，并在连续块之间
    指定重叠。分块对 RAG 系统至关重要，因为它允许更精确地
    检索相关信息。
    
    Args:
        text (str): 要分块的输入文本
        chunk_size (int): 每个块的最大字符大小
        overlap (int): 连续块之间的重叠字符数，
                       用于在块边界之间保持上下文
        
    Returns:
        List[Dict]: 文本块列表，每个包含：
                   - text: 块内容
                   - metadata: 包含位置信息和源类型的字典
    """
    chunks = []
    
    # 使用滑动窗口方法遍历文本
    # 移动 (chunk_size - overlap) 确保块之间的适当重叠
    for i in range(0, len(text), chunk_size - overlap):
        # 提取当前块，受 chunk_size 限制
        chunk_text = text[i:i + chunk_size]
        
        # 只添加非空块
        if chunk_text:
            chunks.append({
                "text": chunk_text,  # 实际文本内容
                "metadata": {
                    "start_pos": i,  # 在原始文本中的起始位置
                    "end_pos": i + len(chunk_text),  # 结束位置
                    "source_type": "document"  # 表示此文本的来源
                }
            })
    
    print(f"创建了 {len(chunks)} 个文本块")
    return chunks

## 简单向量存储实现

In [ ]:
class SimpleVectorStore:
    """
    使用 NumPy 的简单向量存储实现。
    """
    def __init__(self):
        # 初始化列表来存储向量、文本和元数据
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        向向量存储中添加项目。
        
        Args:
            text (str): 文本内容
            embedding (List[float]): 嵌入向量
            metadata (Dict, optional): 附加元数据
        """
        # 将嵌入、文本和元数据追加到各自的列表中
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def add_items(self, items, embeddings):
        """
        向向量存储中添加多个项目。
        
        Args:
            items (List[Dict]): 包含文本和元数据的项目列表
            embeddings (List[List[float]]): 嵌入向量列表
        """
        # 遍历项目和嵌入并将它们添加到存储中
        for i, (item, embedding) in enumerate(zip(items, embeddings)):
            self.add_item(
                text=item["text"],
                embedding=embedding,
                metadata=item.get("metadata", {})
            )
    
    def similarity_search(self, query_embedding, k=5):
        """
        查找与查询嵌入最相似的项目。
        
        Args:
            query_embedding (List[float]): 查询嵌入向量
            k (int): 要返回的结果数量
            
        Returns:
            List[Dict]: 前 k 个最相似的项目
        """
        # 如果存储中没有向量，返回空列表
        if not self.vectors:
            return []
        
        # 将查询嵌入转换为 numpy 数组
        query_vector = np.array(query_embedding)
        
        # 使用余弦相似度计算相似性
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 按相似度排序（降序）
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 返回前 k 个结果
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": float(score)
            })
        
        return results

## 创建嵌入向量

In [ ]:
def create_embeddings(texts, model="text-embedding-3-small"):
    """
    使用 OpenAI 的嵌入模型为文本输入创建向量嵌入。
    
    嵌入是捕获语义含义的文本的密集向量表示，
    允许进行相似性比较。在 RAG 系统中，嵌入对于
    将查询与相关文档块匹配至关重要。
    
    Args:
        texts (str or List[str]): 要嵌入的输入文本。可以是单个字符串
                                  或字符串列表。
        model (str): 要使用的嵌入模型名称。默认为 "text-embedding-3-small"。
        
    Returns:
        List[List[float]]: 如果输入是列表，返回嵌入向量列表。
                          如果输入是单个字符串，返回单个嵌入向量。
    """
    # 通过将单个字符串转换为列表来处理单个字符串和列表输入
    input_texts = texts if isinstance(texts, list) else [texts]
    
    # 分批处理以避免 API 速率限制和负载大小限制
    # OpenAI API 通常对请求大小和速率有限制
    batch_size = 100
    all_embeddings = []
    
    # 处理每批文本
    for i in range(0, len(input_texts), batch_size):
        # 提取当前批次的文本
        batch = input_texts[i:i + batch_size]
        
        # 调用 API 为当前批次生成嵌入
        response = client.embeddings.create(
            model=model,
            input=batch
        )
        
        # 从响应中提取嵌入向量
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)
    
    # 如果原始输入是单个字符串，只返回第一个嵌入
    if isinstance(texts, str):
        return all_embeddings[0]
    
    # 否则返回完整的嵌入列表
    return all_embeddings

## 文档处理流水线

In [ ]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    将文档处理为向量存储。
    
    Args:
        pdf_path (str): PDF 文件路径
        chunk_size (int): 每个块的字符大小
        chunk_overlap (int): 块之间的重叠字符数
        
    Returns:
        SimpleVectorStore: 包含文档块的向量存储
    """
    # 从 PDF 文件中提取文本
    text = extract_text_from_pdf(pdf_path)
    
    # 将提取的文本分割为指定大小和重叠的块
    chunks = chunk_text(text, chunk_size, chunk_overlap)
    
    # 为每个文本块创建嵌入
    print("正在为块创建嵌入向量...")
    chunk_texts = [chunk["text"] for chunk in chunks]
    chunk_embeddings = create_embeddings(chunk_texts)
    
    # 初始化新的向量存储
    vector_store = SimpleVectorStore()
    
    # 将块及其嵌入添加到向量存储中
    vector_store.add_items(chunks, chunk_embeddings)
    
    print(f"创建了包含 {len(chunks)} 个块的向量存储")
    return vector_store

## 相关性评估函数

In [ ]:
def evaluate_document_relevance(query, document):
    """
    评估文档与查询的相关性。
    
    Args:
        query (str): 用户查询
        document (str): 文档文本
        
    Returns:
        float: 相关性分数 (0-1)
    """
    # 定义系统提示，指导模型如何评估相关性
    system_prompt = """
    您是评估文档相关性的专家。
    请评估给定文档与查询的相关性，评分范围为 0 到 1。
    0 表示完全不相关，1 表示完全相关。
    只提供 0 到 1 之间的浮点数分数。
    """
    
    # 定义包含查询和文档的用户提示
    user_prompt = f"查询：{query}\n\n文档：{document}"
    
    try:
        # 向 OpenAI API 发出请求以评估相关性
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # 指定要使用的模型
            messages=[
                {"role": "system", "content": system_prompt},  # 指导助手的系统消息
                {"role": "user", "content": user_prompt}  # 包含查询和文档的用户消息
            ],
            temperature=0,  # 设置响应生成的温度
            max_tokens=5  # 需要非常短的响应
        )
        
        # 从响应中提取分数
        score_text = response.choices[0].message.content.strip()
        # 使用正则表达式在响应中查找浮点值
        score_match = re.search(r'(\d+(\.\d+)?)', score_text)
        if score_match:
            return float(score_match.group(1))  # 将提取的分数作为浮点数返回
        return 0.5  # 如果解析失败，默认为中间值
    
    except Exception as e:
        # 出错时打印错误消息并返回默认值
        print(f"评估文档相关性时出错：{e}")
        return 0.5  # 出错时默认为中间值

## 网络搜索函数

In [ ]:
def duck_duck_go_search(query, num_results=3):
    """
    使用 DuckDuckGo 执行网络搜索。
    
    Args:
        query (str): 搜索查询
        num_results (int): 要返回的结果数量
        
    Returns:
        Tuple[str, List[Dict]]: 合并的搜索结果文本和源元数据
    """
    # 为 URL 编码查询
    encoded_query = quote_plus(query)
    
    # DuckDuckGo 搜索 API 端点（非官方）
    url = f"https://api.duckduckgo.com/?q={encoded_query}&format=json"
    
    try:
        # 执行网络搜索请求
        response = requests.get(url, headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        })
        data = response.json()
        
        # 初始化变量来存储结果文本和源
        results_text = ""
        sources = []
        
        # 如果可用，添加摘要
        if data.get("AbstractText"):
            results_text += f"{data['AbstractText']}\n\n"
            sources.append({
                "title": data.get("AbstractSource", "Wikipedia"),
                "url": data.get("AbstractURL", "")
            })
        
        # 添加相关主题
        for topic in data.get("RelatedTopics", [])[:num_results]:
            if "Text" in topic and "FirstURL" in topic:
                results_text += f"{topic['Text']}\n\n"
                sources.append({
                    "title": topic.get("Text", "").split(" - ")[0],
                    "url": topic.get("FirstURL", "")
                })
        
        return results_text, sources
    
    except Exception as e:
        # 如果主搜索失败，打印错误消息
        print(f"执行网络搜索时出错：{e}")
        
        # 回退到备用搜索 API
        try:
            backup_url = f"https://serpapi.com/search.json?q={encoded_query}&engine=duckduckgo"
            response = requests.get(backup_url)
            data = response.json()
            
            # 初始化变量来存储结果文本和源
            results_text = ""
            sources = []
            
            # 从备用 API 提取结果
            for result in data.get("organic_results", [])[:num_results]:
                results_text += f"{result.get('title', '')}：{result.get('snippet', '')}\n\n"
                sources.append({
                    "title": result.get("title", ""),
                    "url": result.get("link", "")
                })
            
            return results_text, sources
        except Exception as backup_error:
            # 如果备用搜索也失败，打印错误消息
            print(f"备用搜索也失败：{backup_error}")
            return "无法检索搜索结果。", []

In [ ]:
def rewrite_search_query(query):
    """
    重写查询以使其更适合网络搜索。
    
    Args:
        query (str): 原始查询
        
    Returns:
        str: 重写的查询
    """
    # 定义系统提示，指导模型如何重写查询
    system_prompt = """
    您是创建有效搜索查询的专家。
    重写给定的查询，使其更适合网络搜索引擎。
    专注于关键词和事实，删除不必要的词语，使其简洁。
    """
    
    try:
        # 向 OpenAI API 发出请求以重写查询
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # 指定要使用的模型
            messages=[
                {"role": "system", "content": system_prompt},  # 指导助手的系统消息
                {"role": "user", "content": f"原始查询：{query}\n\n重写的查询："}  # 包含原始查询的用户消息
            ],
            temperature=0.3,  # 设置响应生成的温度
            max_tokens=50  # 限制响应长度
        )
        
        # 从响应中返回重写的查询
        return response.choices[0].message.content.strip()
    except Exception as e:
        # 出错时打印错误消息并返回原始查询
        print(f"重写搜索查询时出错：{e}")
        return query  # 出错时返回原始查询

In [ ]:
def perform_web_search(query):
    """
    执行带查询重写的网络搜索。
    
    Args:
        query (str): 原始用户查询
        
    Returns:
        Tuple[str, List[Dict]]: 搜索结果文本和源元数据
    """
    # 重写查询以改善搜索结果
    rewritten_query = rewrite_search_query(query)
    print(f"重写的搜索查询：{rewritten_query}")
    
    # 使用重写的查询执行网络搜索
    results_text, sources = duck_duck_go_search(rewritten_query)
    
    # 返回搜索结果文本和源元数据
    return results_text, sources

## 知识精炼函数

In [ ]:
def refine_knowledge(text):
    """
    从文本中提取和精炼关键信息。
    
    Args:
        text (str): 要精炼的输入文本
        
    Returns:
        str: 从文本中精炼的关键要点
    """
    # 定义系统提示，指导模型如何提取关键信息
    system_prompt = """
    从以下文本中提取关键信息，形成一组清晰、简洁的要点。
    专注于最相关的事实和重要细节。
    将您的响应格式化为项目符号列表，每个要点在新行上以 "• " 开头。
    """
    
    try:
        # 向 OpenAI API 发出请求以精炼文本
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # 指定要使用的模型
            messages=[
                {"role": "system", "content": system_prompt},  # 指导助手的系统消息
                {"role": "user", "content": f"要精炼的文本：\n\n{text}"}  # 包含要精炼文本的用户消息
            ],
            temperature=0.3  # 设置响应生成的温度
        )
        
        # 从响应中返回精炼的关键要点
        return response.choices[0].message.content.strip()
    except Exception as e:
        # 出错时打印错误消息并返回原始文本
        print(f"精炼知识时出错：{e}")
        return text  # 出错时返回原始文本

## 核心 CRAG 流程

In [ ]:
def crag_process(query, vector_store, k=3):
    """
    运行纠错 RAG 流程。
    
    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 包含文档块的向量存储
        k (int): 要检索的初始文档数量
        
    Returns:
        Dict: 包括响应和调试信息的流程结果
    """
    print(f"\n=== 使用 CRAG 处理查询：{query} ===\n")
    
    # 步骤 1：创建查询嵌入并检索文档
    print("正在检索初始文档...")
    query_embedding = create_embeddings(query)
    retrieved_docs = vector_store.similarity_search(query_embedding, k=k)
    
    # 步骤 2：评估文档相关性
    print("正在评估文档相关性...")
    relevance_scores = []
    for doc in retrieved_docs:
        score = evaluate_document_relevance(query, doc["text"])
        relevance_scores.append(score)
        doc["relevance"] = score
        print(f"文档相关性评分：{score:.2f}")
    
    # 步骤 3：根据最佳相关性分数确定行动
    max_score = max(relevance_scores) if relevance_scores else 0
    best_doc_idx = relevance_scores.index(max_score) if relevance_scores else -1
    
    # 跟踪源以进行归属
    sources = []
    final_knowledge = ""
    
    # 步骤 4：执行适当的知识获取策略
    if max_score > 0.7:
        # 情况 1：高相关性 - 直接使用文档
        print(f"高相关性（{max_score:.2f}）- 直接使用文档")
        best_doc = retrieved_docs[best_doc_idx]["text"]
        final_knowledge = best_doc
        sources.append({
            "title": "文档",
            "url": ""
        })
        
    elif max_score < 0.3:
        # 情况 2：低相关性 - 使用网络搜索
        print(f"低相关性（{max_score:.2f}）- 执行网络搜索")
        web_results, web_sources = perform_web_search(query)
        final_knowledge = refine_knowledge(web_results)
        sources.extend(web_sources)
        
    else:
        # 情况 3：中等相关性 - 结合文档与网络搜索
        print(f"中等相关性（{max_score:.2f}）- 结合文档与网络搜索")
        best_doc = retrieved_docs[best_doc_idx]["text"]
        refined_doc = refine_knowledge(best_doc)
        
        # 获取网络结果
        web_results, web_sources = perform_web_search(query)
        refined_web = refine_knowledge(web_results)
        
        # 结合知识
        final_knowledge = f"来自文档：\n{refined_doc}\n\n来自网络搜索：\n{refined_web}"
        
        # 添加源
        sources.append({
            "title": "文档",
            "url": ""
        })
        sources.extend(web_sources)
    
    # 步骤 5：生成最终响应
    print("正在生成最终响应...")
    response = generate_response(query, final_knowledge, sources)
    
    # 返回综合结果
    return {
        "query": query,
        "response": response,
        "retrieved_docs": retrieved_docs,
        "relevance_scores": relevance_scores,
        "max_relevance": max_score,
        "final_knowledge": final_knowledge,
        "sources": sources
    }

## 响应生成

In [ ]:
def generate_response(query, knowledge, sources):
    """
    基于查询和知识生成响应。
    
    Args:
        query (str): 用户查询
        knowledge (str): 用于生成响应的知识
        sources (List[Dict]): 包含标题和 URL 的源列表
        
    Returns:
        str: 生成的响应
    """
    # 格式化源以包含在提示中
    sources_text = ""
    for source in sources:
        title = source.get("title", "未知来源")
        url = source.get("url", "")
        if url:
            sources_text += f"- {title}：{url}\n"
        else:
            sources_text += f"- {title}\n"
    
    # 定义系统提示，指导模型如何生成响应
    system_prompt = """
    您是一个有用的 AI 助手。基于提供的知识对查询生成全面、信息丰富的响应。
    包含所有相关信息，同时保持答案清晰简洁。
    如果知识不能完全回答查询，请承认这一限制。
    在响应末尾包含源归属。
    """
    
    # 定义包含查询、知识和源的用户提示
    user_prompt = f"""
    查询：{query}
    
    知识：
    {knowledge}
    
    来源：
    {sources_text}
    
    请基于这些信息对查询提供信息丰富的响应。
    在响应末尾包含来源。
    """
    
    try:
        # 向 OpenAI API 发出请求以生成响应
        response = client.chat.completions.create(
            model="gpt-4",  # 使用 GPT-4 获得高质量响应
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2
        )
        
        # 返回生成的响应
        return response.choices[0].message.content.strip()
    except Exception as e:
        # 打印错误消息并返回错误响应
        print(f"生成响应时出错：{e}")
        return f"抱歉，我在生成对您查询的响应时遇到了错误：'{query}'。错误是：{str(e)}"

## 评估函数

In [ ]:
def evaluate_crag_response(query, response, reference_answer=None):
    """
    评估 CRAG 响应的质量。
    
    Args:
        query (str): 用户查询
        response (str): 生成的响应
        reference_answer (str, optional): 用于比较的参考答案
        
    Returns:
        Dict: 评估指标
    """
    # 评估标准的系统提示
    system_prompt = """
    您是评估问题响应质量的专家。
    请根据以下标准评估提供的响应：
    
    1. 相关性（0-10）：响应多大程度上直接解决了查询？
    2. 准确性（0-10）：信息的事实正确性如何？
    3. 完整性（0-10）：响应多全面地回答了查询的所有方面？
    4. 清晰度（0-10）：响应多清晰易懂？
    5. 源质量（0-10）：响应引用相关源的效果如何？
    
    将您的评估作为 JSON 对象返回，包含每个标准的分数和每个分数的简要解释。
    还包括一个 "overall_score"（0-10）和您评估的简要 "summary"。
    """
    
    # 包含要评估的查询和响应的用户提示
    user_prompt = f"""
    查询：{query}
    
    要评估的响应：
    {response}
    """
    
    # 如果提供了参考答案，将其包含在提示中
    if reference_answer:
        user_prompt += f"""
    参考答案（用于比较）：
    {reference_answer}
    """
    
    try:
        # 从 GPT-4 模型请求评估
        evaluation_response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"},
            temperature=0
        )
        
        # 解析评估响应
        evaluation = json.loads(evaluation_response.choices[0].message.content)
        return evaluation
    except Exception as e:
        # 处理评估过程中的任何错误
        print(f"评估响应时出错：{e}")
        return {
            "error": str(e),
            "overall_score": 0,
            "summary": "由于错误，评估失败。"
        }

In [ ]:
def compare_crag_vs_standard_rag(query, vector_store, reference_answer=None):
    """
    比较 CRAG 与标准 RAG 对查询的处理。
    
    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 包含文档块的向量存储
        reference_answer (str, optional): 用于比较的参考答案
        
    Returns:
        Dict: 比较结果
    """
    # 运行 CRAG 流程
    print("\n=== 运行 CRAG ===")
    crag_result = crag_process(query, vector_store)
    crag_response = crag_result["response"]
    
    # 运行标准 RAG（直接检索和响应）
    print("\n=== 运行标准 RAG ===")
    query_embedding = create_embeddings(query)
    retrieved_docs = vector_store.similarity_search(query_embedding, k=3)
    combined_text = "\n\n".join([doc["text"] for doc in retrieved_docs])
    standard_sources = [{"title": "文档", "url": ""}]
    standard_response = generate_response(query, combined_text, standard_sources)
    
    # 评估两种方法
    print("\n=== 评估 CRAG 响应 ===")
    crag_eval = evaluate_crag_response(query, crag_response, reference_answer)
    
    print("\n=== 评估标准 RAG 响应 ===")
    standard_eval = evaluate_crag_response(query, standard_response, reference_answer)
    
    # 比较方法
    print("\n=== 比较方法 ===")
    comparison = compare_responses(query, crag_response, standard_response, reference_answer)
    
    return {
        "query": query,
        "crag_response": crag_response,
        "standard_response": standard_response,
        "reference_answer": reference_answer,
        "crag_evaluation": crag_eval,
        "standard_evaluation": standard_eval,
        "comparison": comparison
    }

In [ ]:
def compare_responses(query, crag_response, standard_response, reference_answer=None):
    """
    比较 CRAG 和标准 RAG 响应。
    
    Args:
        query (str): 用户查询
        crag_response (str): CRAG 响应
        standard_response (str): 标准 RAG 响应
        reference_answer (str, optional): 参考答案
        
    Returns:
        str: 比较分析
    """
    # 比较两种方法的系统提示
    system_prompt = """
    您是比较两种响应生成方法的专家评估员：
    
    1. CRAG（纠错 RAG）：一个评估文档相关性并在需要时动态切换到网络搜索的系统。
    2. 标准 RAG：一个基于嵌入相似性直接检索文档并用于响应生成的系统。
    
    基于以下方面比较这两个系统的响应：
    - 准确性和事实正确性
    - 与查询的相关性
    - 答案的完整性
    - 清晰度和组织
    - 源归属质量
    
    解释哪种方法在这个特定查询上表现更好以及原因。
    """
    
    # 包含要比较的查询和响应的用户提示
    user_prompt = f"""
    查询：{query}
    
    CRAG 响应：
    {crag_response}
    
    标准 RAG 响应：
    {standard_response}
    """
    
    # 如果提供了参考答案，将其包含在提示中
    if reference_answer:
        user_prompt += f"""
    参考答案：
    {reference_answer}
    """
    
    try:
        # 从 GPT-4 模型请求比较
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"比较响应时出错：{e}")
        return f"比较失败：{str(e)}"

## 示例使用和评估

以下代码演示了如何使用 CRAG 系统并与标准 RAG 进行比较。

In [ ]:
# 示例使用
if __name__ == "__main__":
    # PDF 文档路径
    pdf_path = "data/AI_Information.pdf"
    
    # 测试查询
    query = "人工智能在医疗保健中的最新应用有哪些？"
    
    # 处理文档并创建向量存储
    vector_store = process_document(pdf_path)
    
    # 运行 CRAG 流程
    crag_result = crag_process(query, vector_store)
    
    print(f"查询：{crag_result['query']}")
    print(f"最大相关性分数：{crag_result['max_relevance']:.2f}")
    print(f"检索的文档数量：{len(crag_result['retrieved_docs'])}")
    print(f"源数量：{len(crag_result['sources'])}")
    print(f"CRAG 响应：{crag_result['response']}")
    
    # 比较 CRAG 与标准 RAG
    comparison_result = compare_crag_vs_standard_rag(query, vector_store)
    print(f"\n比较分析：{comparison_result['comparison']}")

## 总结

纠错 RAG (CRAG) 通过以下方式改进了传统的 RAG 系统：

1. **动态相关性评估**：在使用检索到的文档之前评估其相关性
2. **自适应知识源选择**：根据相关性分数动态选择最佳知识源
3. **网络搜索集成**：当本地知识不足时，无缝集成网络搜索作为后备
4. **知识精炼**：提取和精炼关键信息以提高响应质量
5. **多源知识融合**：在适当时结合来自多个源的信息

这种方法特别适用于：
- 需要最新信息的查询
- 本地文档可能不完整的情况
- 需要高准确性和相关性的应用
- 需要透明源归属的系统

CRAG 代表了 RAG 系统设计中的重要进步，为构建更智能、更可靠的问答系统提供了框架，能够动态适应不同的查询需求和知识可用性。